In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uzbek-news-dataset/balanced_news_dataset_100k.csv


In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/uzbek-news-dataset/balanced_news_dataset_100k.csv')
df.head()

,text,label
0,U-Report: O‘zbekistonda yangi innovatsion loyi...,fan va texnika
1,Bolalikda o‘qilgan fantastik kitoblar yoxud 20...,fan va texnika
2,Kanadalik erkak 14 yildan buyon o‘yinchoq eksk...,fan va texnika
3,Samsung Galaxy S21 Ultra eng yaxshi kamerofonl...,fan va texnika
4,Meizu’ning ilk romsiz smartfoni suratlari payd...,fan va texnika


In [3]:
df.shape, df['label'].value_counts()

((584276, 2),
 label
 jahon             100000
 jamiyat           100000
 qonunchilik       100000
 o'zbekiston       100000
 sport             100000
 fan va texnika     84276
 Name: count, dtype: int64)

## Preprocessing the data

In [4]:
import re

def preprocess_uzbek(text):
    text = text.lower()  # lowercase
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = text.strip()  # remove leading/trailing spaces
    text = re.sub(r"\s+", " ", text)  # replace multiple spaces with single
    return text

In [5]:
df['text'] = df['text'].apply(preprocess_uzbek)


In [6]:
df.tail(5)

,text,label
584271,18 avgust 2 sentabr kunlari indoneziyaning jak...,sport
584272,futbol boyicha yevropa chempionatlari tarixida...,sport
584273,poytaxtimizdagi lotte city hotel tashkent pala...,sport
584274,video daniyada futbol muxlislari maydonga olik...,sport
584275,sport fidoyisi koplab chempionlarni tarbiyalag...,sport


In [7]:
# Shuffle the entire dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the first few rows after shuffling
print(df.head())


                                                text           label
0  oq uyning milliy xavfsizlik masalalari boyicha...           jahon
1  apple 32 gb xotirali yangi ipad pro planshetin...  fan va texnika
2  penzada roy bergan dahshatli ythda 4 kishi hal...           jahon
3  11iyun dunyo okeanlari tadqiqotchisi fransuz d...           jahon
4  bugun 10 sentyabr kuni ozbekiston respublikasi...         jamiyat


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-04-11 16:25:05.961460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744388706.428572      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744388706.566516      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# # Sample text data
# sample_texts = ['O\'zbekiston sport sohasida yirik o\'zgarishlar yuz berdi.',
#                 'Jahon yangiliklari va sport musobaqalari haqida ma\'lumotlar.']

# # Tokenizer instance
# tokenizer = Tokenizer(num_words=5000)  # Limit vocab size to 5000 words
# tokenizer.fit_on_texts(sample_texts)  # Fit tokenizer on the sample text data

# # Convert the texts to sequences of integers
# sequences = tokenizer.texts_to_sequences(sample_texts)

# # Print the result
# print(f"Tokenized Sequences: {sequences}")


In [10]:
# # Example tokenized sequences
# sequences = [[2, 1, 3, 4, 5, 6, 7], [8, 9, 10, 1, 11, 12, 13]]

# # Create the reverse word_index
# reverse_word_index = {index: word for word, index in tokenizer.word_index.items()}

# # Convert the sequences back to words
# sequences_as_words = [[reverse_word_index.get(index, '') for index in sequence] for sequence in sequences]

# # Print the tokenized text as words
# print(sequences_as_words)


In [11]:
text_lengths = df['text'].apply(lambda x: len(x.split()))

# Check the distribution (mean, max, etc.)
print(f"Mean Length: {text_lengths.mean()}")
print(f"Max Length: {text_lengths.max()}")
print(f"Min Length: {text_lengths.min()}")

Mean Length: 300.77756916251906
Max Length: 8012
Min Length: 0


In [12]:
# Assuming 'df' is your DataFrame and 'text' is the column with the text data
tokenizer = Tokenizer(num_words=5000)  # Limit vocab size to top 5000 words
tokenizer.fit_on_texts(df['text'])  # Fit tokenizer on the 'text' column

# Convert texts to sequences of integers
sequences = tokenizer.texts_to_sequences(df['text'])

# Pad sequences to ensure all are the same length
max_length = 300  # You can adjust this value based on your data and model requirements
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# View the tokenized and padded sequences
print(padded_sequences)

[[1316 4330   75 ...    0    0    0]
 [ 824  914 1552 ...    0    0    0]
 [1647  266   99 ...    0    0    0]
 ...
 [ 408 1014 1176 ...    0    0    0]
 [3567 3500 1309 ...    0    0    0]
 [1360 1777   29 ...    0    0    0]]


In [13]:
df.tail(3)

,text,label
584273,ohangaron tumani 1umumtalim maktabining 11sinf...,o'zbekiston
584274,yurak kochirish operatsiyasi yanvar oyida meri...,jahon
584275,shvetsiyalik olimlar tadqiqot otkazishdi u qis...,jahon


In [14]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])  # Replace 'category' with your column name


In [15]:
df['label'].value_counts()

label
1    100000
2    100000
4    100000
3    100000
5    100000
0     84276
Name: count, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

# Assuming you have labels in a column named 'label' in your dataframe
X = padded_sequences  # Features (tokenized and padded text)
y = df['label'].values  # Labels (your target variable)

# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (525848, 300)
Validation data shape: (58428, 300)


In [17]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 76.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2

In [24]:
y_train

array([4, 4, 0, ..., 3, 1, 2])

## Model 1

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

num_classes = len(np.unique(y_train))  # Should be 5 in your case

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=300))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))  # Use softmax for multi-class

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, X.shape[1]))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 300, 100)            │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 128)                 │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 618,022 (2.36 MB)

 Trainable params: 618,022 (2.36 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# evaluating to see reaction of model to values
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print(f"Initial loss: {loss:.4f}")
print(f"Initial accuracy: {accuracy:.4f}")


16433/16433 ━━━━━━━━━━━━━━━━━━━━ 86s 5ms/step - accuracy: 0.1767 - loss: 1.7917
Initial loss: 1.7917
Initial accuracy: 0.1761


In [28]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val))

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")

Epoch 1/5
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 126s 15ms/step - accuracy: 0.5241 - loss: 1.0970 - val_accuracy: 0.8433 - val_loss: 0.4056
Epoch 2/5
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 126s 15ms/step - accuracy: 0.8556 - loss: 0.3845 - val_accuracy: 0.8753 - val_loss: 0.3267
Epoch 3/5
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 127s 15ms/step - accuracy: 0.8819 - loss: 0.3109 - val_accuracy: 0.8838 - val_loss: 0.3010
Epoch 4/5
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 126s 15ms/step - accuracy: 0.8981 - loss: 0.2678 - val_accuracy: 0.8910 - val_loss: 0.2812
Epoch 5/5
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 125s 15ms/step - accuracy: 0.9126 - loss: 0.2322 - val_accuracy: 0.8957 - val_loss: 0.2697
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8953 - loss: 0.2698
Validation Loss: 0.26973193883895874
Validation Accuracy: 0.8957006931304932


## Model 2

In [30]:
model2 = Sequential()
model2.add(Embedding(input_dim=5000, output_dim=100, input_length=300))
model2.add(LSTM(128, return_sequences=False))
model2.add(Dropout(0.4))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(num_classes, activation='softmax'))  # Use softmax for multi-class

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.build(input_shape=(None, X.shape[1]))
model2.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 300, 100)            │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 625,894 (2.39 MB)

 Trainable params: 625,894 (2.39 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64, callbacks=[early_stop])

val_loss, val_acc = model2.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")

Epoch 1/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 133s 16ms/step - accuracy: 0.4678 - loss: 1.1861 - val_accuracy: 0.8312 - val_loss: 0.4151
Epoch 2/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 0.8528 - loss: 0.3954 - val_accuracy: 0.8752 - val_loss: 0.3222
Epoch 3/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 0.8807 - loss: 0.3185 - val_accuracy: 0.8809 - val_loss: 0.3016
Epoch 4/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 0.8954 - loss: 0.2786 - val_accuracy: 0.8879 - val_loss: 0.2898
Epoch 5/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 0.9072 - loss: 0.2472 - val_accuracy: 0.8915 - val_loss: 0.2818
Epoch 6/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 0.9190 - loss: 0.2184 - val_accuracy: 0.8967 - val_loss: 0.2778
Epoch 7/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 0.9288 - loss: 0.1944 - val_accuracy: 0.9000 - val_loss: 0.2723
Epoch 8/15
8217/8217 ━━━━━━━━━━━━━━━━━━━━ 128s 16ms/step - accuracy: 

In [32]:
# Save the model to a file (in H5 format)
model2.save('uzbek_news_classification_model.h5')  # Replace 'your_model_name.h5' with your preferred file name

In [51]:
# import numpy as np
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import load_model
# import re

# # # Load the trained model
# # model2 = load_model('your_model_name.h5')

# # Assume you have a function to clean the text (you can define your own cleaning logic)
# def clean_text(text):
#     # Example cleaning logic (remove punctuation, lowercase, etc.)
#     text = text.lower()  # lowercase
#     text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
#     text = text.strip()  # remove leading/trailing spaces
#     text = re.sub(r"\s+", " ", text)  # replace multiple spaces with single
#     return text

# # New text to test
# new_text = ("«Tog‘ oldi va tog‘li hududlarda istiqomat qiluvchi fuqarolarni, dam oluvchilarni hamda tog‘li hududlarda harakatlanuvchi haydovchilarni ehtiyotkorlik choralarini ko‘rishlarini so‘raymiz. Respublika hududi bo‘yicha yomg‘ir suvlari to‘planishi ehtimoli bor, bu esa hududlarni suv bosishiga olib kelishi mumkin», — deyiladi xabarda.")


# # Clean the new text
# cleaned_text = clean_text(new_text)

# # Tokenize and pad the new text (same as how you preprocessed the training data)
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts([cleaned_text])  # Fit on the new text (or you could use the same tokenizer from training)
# sequence = tokenizer.texts_to_sequences([cleaned_text])
# padded_sequence = pad_sequences(sequence, maxlen=300, padding='post', truncating='post')

# # Make prediction with the model
# prediction = model2.predict(padded_sequence)

# # Print prediction (for multi-class, we will use argmax to get the class with the highest probability)
# predicted_class = np.argmax(prediction, axis=1)

# print(f"Predicted class: {predicted_class[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: 4
